In [5]:
import simpy
import numpy as np
import pandas as pd 

class Server:
    def __init__(self, env, speed=1, latency=0, cost=1):
        self.env = env
        self.machine = simpy.Resource(env, capacity=1)
        self.latest_task_end_time = 0
        self.speed = speed
        self.latency = latency
        self.cost = cost  # New attribute for cost
        self.expected_end_time = 0
        self.total_processing_time = 0
        self.total_cost = 0  # New attribute for total cost

    def process(self, task):
        with self.machine.request() as req:
            yield req
            yield self.env.timeout((task.duration / self.speed) + self.latency)
            self.latest_task_end_time = self.env.now
            self.expected_end_time = self.env.now + (task.duration / self.speed) + self.latency
            self.total_processing_time += (task.duration / self.speed) + self.latency
            self.total_cost += self.cost * (task.duration / self.speed) + self.latency  # Calculate total cost

class Task:
    def __init__(self, duration):
        self.duration = duration

MAX_COST = 350

class FitnessFunction:
    def __init__(self, tasks, servers):
        self.tasks = tasks
        self.servers = servers
        self.makespans = {task_size: [] for task_size in tasks}
        self.average_cost = 0
        self.average_utilization = 0

    def __call__(self, solution):
        env = simpy.Environment()
        servers = [Server(env, speed=speed, latency=latency, cost=cost) for speed, latency, cost in self.servers]  # Initialize servers with speed, latency, and cost
        task_objects = []
        
        for task_duration, server_index in zip(self.tasks, solution):
            task = Task(task_duration)
            task_objects.append(task)  # Keep track of task objects
            server = servers[int(server_index)]
            env.process(server.process(task))

        env.run()

        # Store makespan for each task size
        for task in task_objects:
            self.makespans[task.duration].append(server.latest_task_end_time)

        makespan = max(server.latest_task_end_time for server in servers)
        total_cost = sum(server.total_cost for server in servers)  # Corrected cost calculation

        self.average_cost = total_cost / len(self.tasks)
        total_utilization = sum(server.total_processing_time / makespan for server in servers)
        self.average_utilization = total_utilization / len(self.servers)

        lambda_factor = 0.01  # hyperparameter that you can tune
        return makespan + lambda_factor * total_cost  # new fitness

    def get_average_cost(self):
        return self.average_cost

    def get_average_utilization(self):
        return self.average_utilization

class Flower:
    def __init__(self, decision_variables, lower_bound, upper_bound):
        self.position = np.random.uniform(
            low=lower_bound, high=upper_bound, size=decision_variables)
        self.fitness = np.inf


class SimulatedAnnealing:
    def __init__(self, fitness_function, initial_solution, initial_temperature, final_temperature, cooling_rate):
        self.fitness_function = fitness_function
        self.current_solution = initial_solution
        self.current_fitness = self.fitness_function(self.current_solution)
        self.best_solution = self.current_solution
        self.best_fitness = self.current_fitness
        self.temperature = initial_temperature
        self.final_temperature = final_temperature
        self.cooling_rate = cooling_rate

    def accept(self, candidate_fitness):
        if candidate_fitness < self.current_fitness:
            return True
        else:
            delta = self.current_fitness - candidate_fitness
            probability = np.exp(delta / self.temperature)
            return np.random.rand() < probability

    def cool_down(self):
        if self.temperature > self.final_temperature:
            self.temperature *= self.cooling_rate


class FlowerPollination:
    def __init__(self, fitness_function, population_size=100, decision_variables=10, lower_bound=0, upper_bound=1, generations=1000, switch_prob=0.8, gamma=0.1, beta=1.5):
        self.population = [Flower(
            decision_variables, lower_bound, upper_bound) for _ in range(population_size)]
        self.global_best = None
        self.fitness_function = fitness_function
        self.population_size = population_size
        self.decision_variables = decision_variables
        self.lower_bound = lower_bound
        self.upper_bound = upper_bound
        self.generations = generations
        self.switch_prob = switch_prob
        self.gamma = gamma
        self.beta = beta

    def levy_flight(self, beta):
        sigma = (np.math.gamma(1 + beta) * np.sin(np.pi * beta / 2) /
                 (np.math.gamma((1 + beta) / 2) * beta * 2 ** ((beta - 1) / 2))) ** (1 / beta)
        u = np.random.normal(0, sigma, size=self.decision_variables)
        v = np.random.normal(0, 1, size=self.decision_variables)
        step = u / abs(v) ** (1 / beta)
        return step

    def global_pollination(self, flower):
        step_size = self.gamma * self.levy_flight(self.beta)
        new_position = flower.position + step_size * \
            (flower.position - self.global_best.position)
        return new_position

    def local_pollination(self, flower):
        flower_j = np.random.choice(self.population)
        epsilon = np.random.uniform(
            low=0, high=1, size=self.decision_variables)
        new_position = flower.position + epsilon * \
            (flower_j.position - flower.position)
        return new_position

    def pollination(self, flower):
        r = np.random.uniform(low=0, high=1)
        if r < self.switch_prob:
            new_position = self.global_pollination(flower)
        else:
            new_position = self.local_pollination(flower)
        return np.clip(new_position, self.lower_bound, self.upper_bound)

    def find_global_best(self):
        for flower in self.population:
            if flower.fitness < self.global_best.fitness:
                self.global_best = flower

    def optimize(self):
        self.global_best = self.population[0]
        self.global_best.fitness = self.fitness_function(self.global_best.position)
        sa = SimulatedAnnealing(self.fitness_function, self.global_best.position, initial_temperature=10, final_temperature=0.001, cooling_rate=0.9)

        for g in range(self.generations):
            for flower in self.population:
                flower.fitness = self.fitness_function(flower.position)
                self.find_global_best()

                new_position = self.pollination(flower)
                new_fitness = self.fitness_function(new_position)

                if sa.accept(new_fitness):
                    flower.position = new_position
                    flower.fitness = new_fitness

            sa.cool_down()  # cool down the temperature after each generation
            self.find_global_best()
            #print(f"Generation: {g}, Best fitness: {self.global_best.fitness}")

        return self.global_best
cloud_servers = [(1, 0.1, 1), (2, 0.1, 1), (3, 0.1, 1), (4, 0.1, 1), (5, 0.1, 1)]  # For cloud servers, cost is 1
edge_servers = [(1, 0.01, 0.5), (2, 0.01, 0.5), (3, 0.01, 0.5), (4, 0.01, 0.5), (5, 0.01, 0.5)]  # For edge servers, cost is 0.5

task_sizes = [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
# Run your optimization for both cloud and edge servers.

# Initialize DataFrame
columns = ['Task Size', 'Server Type', 'Average Makespan', 'Average Cost', 'Average Utilization']
df = pd.DataFrame(columns=columns)

# Optimization and DataFrame appending loop
for task_size in task_sizes:
    # Optimization for cloud servers
    print(f"Optimizing for cloud servers with task size: {task_size}")
    fitness_function = FitnessFunction([task_size], cloud_servers)
    fp = FlowerPollination(fitness_function, decision_variables=1, lower_bound=0, upper_bound=len(cloud_servers) - 1, generations=100)
    best_flower = fp.optimize()
    average_cost = fitness_function.get_average_cost()
    average_utilization = fitness_function.get_average_utilization()*task_size/9.5
    df.loc[len(df)] = [task_size, 'Cloud', np.mean(fitness_function.makespans[task_size]), average_cost, average_utilization]
    
    # Optimization for edge servers
    print(f"Optimizing for edge servers with task size: {task_size}")
    fitness_function = FitnessFunction([task_size], edge_servers)
    fp = FlowerPollination(fitness_function, decision_variables=1, lower_bound=0, upper_bound=len(edge_servers) - 1, generations=100)
    best_flower = fp.optimize()
    average_cost = fitness_function.get_average_cost()
    average_utilization = fitness_function.get_average_utilization()*task_size/10.5
    df.loc[len(df)] = [task_size, 'Edge', np.mean(fitness_function.makespans[task_size]), average_cost, average_utilization]

    # Print the DataFrame after each optimization cycle
    print("Current DataFrame:")
    print(df)

# Save the DataFrame as a CSV file
df.to_csv('simulation_results.csv', index=False)


Optimizing for cloud servers with task size: 100
Optimizing for edge servers with task size: 100
Current DataFrame:
   Task Size Server Type  Average Makespan  Average Cost  Average Utilization
0        100       Cloud         34.229254     33.433333             2.105263
1        100        Edge         42.000551     16.676667             1.904762
Optimizing for cloud servers with task size: 200
Optimizing for edge servers with task size: 200
Current DataFrame:
   Task Size Server Type  Average Makespan  Average Cost  Average Utilization
0        100       Cloud         34.229254     33.433333             2.105263
1        100        Edge         42.000551     16.676667             1.904762
2        200       Cloud         62.031640     50.100000             4.210526
3        200        Edge         71.178217     25.010000             3.809524
Optimizing for cloud servers with task size: 300
Optimizing for edge servers with task size: 300
Current DataFrame:
   Task Size Server Type  Av